In [47]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [48]:
df2 = pd.read_csv('deleted.csv')

In [49]:
columns_name=df2.columns
print(columns_name)
print(len(columns_name))
df2.fillna(32,inplace=True)

Index(['game_id', 'season', 'num_ot', 'team1_pt_school_ncaa',
       'team1_pt_overall_ncaa', 'team1_pt_school_s16', 'team1_pt_overall_s16',
       'team1_pt_school_ff', 'team1_pt_overall_ff',
       'team1_pt_career_school_wins', 'team1_pt_career_school_losses',
       'team1_pt_career_overall_wins', 'team1_pt_career_overall_losses',
       'team1_pt_team_season_wins', 'team1_pt_team_season_losses',
       'team1_pt_coach_season_wins', 'team1_pt_coach_season_losses',
       'team2_pt_school_ncaa', 'team2_pt_overall_ncaa', 'team2_pt_school_s16',
       'team2_pt_overall_s16', 'team2_pt_school_ff', 'team2_pt_overall_ff',
       'team2_pt_career_school_wins', 'team2_pt_career_school_losses',
       'team2_pt_career_overall_wins', 'team2_pt_career_overall_losses',
       'team2_pt_team_season_wins', 'team2_pt_team_season_losses',
       'team2_pt_coach_season_wins', 'team2_pt_coach_season_losses',
       'team1_ap_final', 'team1_ap_preseason', 'team1_coaches_before_final',
       'team1_c

In [50]:
df2['team1_seasonwin'] = df2['team1_pt_coach_season_wins']/(df2['team1_pt_coach_season_wins']+df2['team1_pt_coach_season_losses'])
df2['team2_seasonwin'] = df2['team2_pt_coach_season_wins']/(df2['team2_pt_coach_season_wins']+df2['team2_pt_coach_season_losses'])
df2['team1_overallwin'] = df2['team1_pt_career_school_wins']/(df2['team1_pt_career_school_wins']+df2['team1_pt_career_school_losses'])
df2['team2_overallwin'] = df2['team2_pt_career_school_wins']/(df2['team2_pt_career_school_wins']+df2['team2_pt_career_school_losses'])
#np.mean(df2['team1_seasonwin'] ), np.max(df2['team1_seasonwin'] ), np.mean(df2['team2_seasonwin']), np.max(df2['team2_seasonwin'])
#np.mean(df2['team1_overallwin']), np.max(df2['team1_overallwin']),np.mean(df2['team2_overallwin']), np.max(df2['team2_overallwin'])
df2['adjoe_diff'] = df2['team1_adjoe'] - df2['team2_adjoe']
df2['adjde_diff'] = df2['team1_adjde'] - df2['team2_adjde']
df3 = df2.drop(columns=['team1_adjoe','team2_adjoe','team1_adjde','team2_adjde','team1_pt_coach_season_wins',\
                       'team2_pt_coach_season_wins','team1_pt_career_school_wins','team2_pt_career_school_wins'])

In [51]:
columns_name=df3.columns
print(columns_name)
print(len(columns_name))
print(len(df3))

Index(['game_id', 'season', 'num_ot', 'team1_pt_school_ncaa',
       'team1_pt_overall_ncaa', 'team1_pt_school_s16', 'team1_pt_overall_s16',
       'team1_pt_school_ff', 'team1_pt_overall_ff',
       'team1_pt_career_school_losses', 'team1_pt_career_overall_wins',
       'team1_pt_career_overall_losses', 'team1_pt_team_season_wins',
       'team1_pt_team_season_losses', 'team1_pt_coach_season_losses',
       'team2_pt_school_ncaa', 'team2_pt_overall_ncaa', 'team2_pt_school_s16',
       'team2_pt_overall_s16', 'team2_pt_school_ff', 'team2_pt_overall_ff',
       'team2_pt_career_school_losses', 'team2_pt_career_overall_wins',
       'team2_pt_career_overall_losses', 'team2_pt_team_season_wins',
       'team2_pt_team_season_losses', 'team2_pt_coach_season_losses',
       'team1_ap_final', 'team1_ap_preseason', 'team1_coaches_before_final',
       'team1_coaches_preseason', 'team2_ap_final', 'team2_ap_preseason',
       'team2_coaches_before_final', 'team2_coaches_preseason', 'team1_fg2pct

In [52]:
#extract season 2017 and 2018 data
d2017 = df3.where(df3['season'] == 2017).dropna(how = 'all')
d2018 = df3.where(df3['season'] == 2018).dropna(how = 'all')
t= [d2017,d2018]
used_for_test = pd.concat(t)

In [54]:
test_result = used_for_test['label']
lst = ['game_id', 'season','label','shuffled']
rawdata_for_test = used_for_test.copy()
rawdata_for_test = rawdata_for_test.drop(columns = lst)

In [55]:
# get our training set
df3 = df3.where(df3['season'] != 2017).dropna(how = 'all')
df3 = df3.where(df3['season'] != 2018).dropna(how = 'all')
print(len(df3))

971


In [56]:
result = df3['label']
lst = ['game_id', 'season','label','shuffled'] ##we need these features, but they are not put in feature selecyion
rawData = df3.copy() 
rawData = rawData.drop(columns=lst)

In [57]:
for column in rawData:
    #print(str(type(rawData[column][0]))[8:11])
    if str(type(rawData[column][0]))[8:11] == 'str':
        del rawData[column]
print(len(rawData.columns))

82


In [58]:
rawData = rawData.dropna(axis=1)
#len(rawData.columns)
print(len(rawData.columns))

82


In [59]:
#L1-based feature selection
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

In [60]:
print("rawData 共有 %s 个特征"%rawData.shape[1])
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(rawData, result)

rawData 共有 82 个特征


In [61]:
model = SelectFromModel(lsvc, prefit=True)
data_lsvc = model.transform(rawData)
print("data_lsvc 共有 %s 个特征"%data_lsvc.shape[1])

data_lsvc 共有 31 个特征


In [62]:
data_lsvc.shape

(971, 31)

In [63]:
df_lsvc= pd.DataFrame(data_lsvc)
df_lsvc.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,317.0,210.0,9.0,0.0,18.0,16.0,18.0,16.0,18.0,32.0,...,74.4032,10.5144,36.6570,54.5135,76.0664,103.4213,99.6370,144.611318,1.2563,6.7615
1,28.0,63.0,11.0,0.0,49.0,149.0,73.0,26.0,7.0,32.0,...,73.5782,7.1637,32.7925,50.5495,71.2357,100.2897,98.4183,88.820532,-14.6285,6.0493
2,62.0,52.0,10.0,10.0,144.0,656.0,239.0,22.0,9.0,32.0,...,73.0769,8.4934,31.9731,56.4706,63.2345,105.2163,96.0965,697.542289,-11.9343,0.7442
3,79.0,41.0,11.0,12.0,167.0,635.0,226.0,29.0,3.0,32.0,...,68.9845,7.4692,26.1916,57.1821,68.2128,100.2384,94.6587,191.666515,-23.2367,12.2250
4,704.0,278.0,8.0,0.0,5.0,27.0,5.0,27.0,5.0,20.0,...,73.1034,5.5978,32.9782,48.4721,68.8801,103.9347,92.8097,1066.404958,-9.4416,-4.3022


In [64]:
#df_lsvc.to_csv("df_lsvc.csv",index=True,sep=',')

In [65]:
#Tree-based feature selection
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier()

In [66]:
etc = etc.fit(rawData, result)

In [67]:
etc.feature_importances_

array([0.00625301, 0.01057721, 0.00763254, 0.00818295, 0.00895662,
       0.00777785, 0.00676052, 0.01113793, 0.00856738, 0.00835356,
       0.00962438, 0.00751811, 0.01080381, 0.00869508, 0.00806778,
       0.01133044, 0.01026341, 0.006019  , 0.00758849, 0.01037355,
       0.01258201, 0.01129216, 0.00895196, 0.00837902, 0.00690096,
       0.02144284, 0.00968073, 0.05861863, 0.01353333, 0.02205288,
       0.01579155, 0.03235037, 0.01275816, 0.01053838, 0.01311731,
       0.00962222, 0.0082519 , 0.00814038, 0.00969378, 0.00857638,
       0.01034691, 0.01097714, 0.0107546 , 0.01007561, 0.00802511,
       0.01063722, 0.00817125, 0.01334341, 0.00944336, 0.01252155,
       0.01253823, 0.0109284 , 0.00679204, 0.01211228, 0.01011626,
       0.01018071, 0.0092976 , 0.00743202, 0.00902153, 0.0150724 ,
       0.01164246, 0.00725779, 0.01002886, 0.01202764, 0.0078551 ,
       0.00858026, 0.01423861, 0.01205801, 0.00761484, 0.00950688,
       0.01163539, 0.01431677, 0.01336299, 0.04736932, 0.04128

In [68]:
model = SelectFromModel(etc, prefit=True)

In [69]:
data_etc = model.transform(rawData)

In [70]:
data_etc.shape

(971, 22)

In [71]:
df_etc= pd.DataFrame(data_etc)
df_etc.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,16.0,32.0,32.0,32.0,32.0,32.0,32.0,32.0,37.5556,45.6127,...,0.0778,67.7359,0.324280,0.468773,0.352264,0.0,144.611318,0.470588,1.2563,6.7615
1,149.0,32.0,32.0,32.0,8.0,24.0,8.0,32.0,33.9114,50.6579,...,0.1053,69.9001,0.411542,0.878465,0.088220,13.0,88.820532,0.623077,-14.6285,6.0493
2,656.0,32.0,32.0,32.0,7.0,32.0,8.0,32.0,40.8247,50.0000,...,0.0835,72.8207,0.707177,0.900485,0.210667,11.0,697.542289,0.764321,-11.9343,0.7442
3,635.0,32.0,32.0,32.0,1.0,1.0,1.0,32.0,30.5603,57.4329,...,0.1297,75.1685,0.367453,0.969660,0.017852,15.0,191.666515,0.770919,-23.2367,12.2250
4,27.0,20.0,22.0,32.0,32.0,32.0,32.0,32.0,32.7451,50.7947,...,0.0974,66.1658,0.809638,0.871998,0.384359,-3.0,1066.404958,0.843750,-9.4416,-4.3022


In [72]:
#df_etc.to_csv("df_etc.csv",index=True,sep=',')

In [73]:
#Pearson correlation linear feature selection
categories = []
categories = rawData.columns

In [74]:
data_pearson = rawData

In [75]:
import scipy
for cat in categories:
    print ('The Pearson correlation between the result and', cat, 
           'is :',scipy.stats.pearsonr(rawData[cat], result))

The Pearson correlation between the result and num_ot is : (0.021895042572756103, 0.49557319239765196)
The Pearson correlation between the result and team1_pt_school_ncaa is : (0.16264189386974257, 3.4782402617915773e-07)
The Pearson correlation between the result and team1_pt_overall_ncaa is : (0.2012427058843505, 2.488979798235356e-10)
The Pearson correlation between the result and team1_pt_school_s16 is : (0.16977762165432433, 1.0247008265533668e-07)
The Pearson correlation between the result and team1_pt_overall_s16 is : (0.21139567726599603, 2.8498408658069337e-11)
The Pearson correlation between the result and team1_pt_school_ff is : (0.14235719936035182, 8.469539007336175e-06)
The Pearson correlation between the result and team1_pt_overall_ff is : (0.17575092600980718, 3.538539222136024e-08)
The Pearson correlation between the result and team1_pt_career_school_losses is : (0.025883990191665377, 0.42043674220504845)
The Pearson correlation between the result and team1_pt_career_o

In [76]:
for cat in categories:
    if scipy.stats.pearsonr(rawData[cat], result)[1]>0.05:# p value low means correlation is strong
        del data_pearson[cat]
    elif scipy.stats.pearsonr(rawData[cat], result)[0]<0.3:
        del data_pearson[cat]

In [77]:
data_pearson.head()
#data_pearson.to_csv("pearson.csv",index=True,sep=',')

,team2_ap_final,team2_coaches_before_final,exp_win1,log5,adjoe_diff
0,32.0,32.0,0.324280,0.352264,1.2563
1,8.0,8.0,0.411542,0.088220,-14.6285
2,7.0,8.0,0.707177,0.210667,-11.9343
3,1.0,1.0,0.367453,0.017852,-23.2367
4,32.0,32.0,0.809638,0.384359,-9.4416


In [79]:
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
std = StandardScaler()

In [120]:
feature_left = []
def feature_selection_for_test(df_train,df_test):
    for i in df_train.columns:
        feature_left.append(i)
    df_test = df_test.loc[:,feature_left]
    return df_test

In [121]:
df_test = feature_selection_for_test(data_pearson,rawdata_for_test)
df_test.head()

,team2_ap_final,team2_coaches_before_final,exp_win1,log5,adjoe_diff
485,32.0,32.0,0.528796,0.659920,5.0529
486,7.0,7.0,0.491105,0.075560,-14.6400
487,18.0,19.0,0.624737,0.149454,-11.5380
488,17.0,10.0,0.784602,0.157369,-7.8200
489,23.0,25.0,0.805661,0.259006,-5.0670


In [122]:
def ensemble_model_score(df_for_train,df_for_test,train_label,test_label):
    clf1 = LogisticRegression(random_state=1)
    clf2 = RFC= RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
        max_depth=4, max_features='auto', max_leaf_nodes=None,
        min_impurity_decrease=0,
        min_samples_leaf=5, min_samples_split=2,
        min_weight_fraction_leaf=0.0, n_estimators=3000, n_jobs=-1,
        oob_score=True, random_state=0, verbose=0, warm_start=False)
    clf3 = svc = SVC(C=5, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, probability=True, tol=0.0001, 
          cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', 
          random_state=None)
    eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')
    X_train = df_for_train
    X_test = df_for_test
    y_train, y_test = train_label, test_label
    for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
        scores = cross_val_score(clf,X_train,y_train, cv=5, scoring='accuracy')
        print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [123]:
import numpy as np
ensemble_model_score(data_pearson,df_test,result,test_result)

Accuracy: 0.69 (+/- 0.03) [Logistic Regression]
Accuracy: 0.69 (+/- 0.03) [Random Forest]
Accuracy: 0.66 (+/- 0.03) [naive Bayes]
Accuracy: 0.69 (+/- 0.03) [Ensemble]


In [126]:
def ensemble_model(df_for_train,df_for_test,train_label,test_label):
    X_train = df_for_train
    X_test = df_for_test
    y_train, y_test = train_label, test_label
    clf1 = LogisticRegression(random_state=1)
    clf2 = RFC= RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
        max_depth=4, max_features='auto', max_leaf_nodes=None,
        min_impurity_decrease=0,
        min_samples_leaf=5, min_samples_split=2,
        min_weight_fraction_leaf=0.0, n_estimators=3000, n_jobs=-1,
        oob_score=True, random_state=0, verbose=0, warm_start=False)
    clf3 = svc = SVC(C=5, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, probability=True, tol=0.0001, 
          cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', 
          random_state=None)
    eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')
    params = {'rf__max_depth': [3, 5]}
    grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
    grid = grid.fit(X_train,y_train)
    print('training log_loss:',log_loss(train_label,grid.predict_proba(df_for_train)))
    print('testing log_loss:',log_loss(test_label,grid.predict_proba(df_for_test)))
    predict_proba=grid.predict_proba(df_for_test)
    print(predict_proba)

In [127]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
ensemble_model(data_pearson,df_test,result,test_result)

training log_loss: 0.5005741939185512
testing log_loss: 0.557323718644141
[[0.34852114 0.65147886]
 [0.85325292 0.14674708]
 [0.74210437 0.25789563]
 [0.81095101 0.18904899]
 [0.64349247 0.35650753]
 [0.38270466 0.61729534]
 [0.77882141 0.22117859]
 [0.81955611 0.18044389]
 [0.67756743 0.32243257]
 [0.60477066 0.39522934]
 [0.77468721 0.22531279]
 [0.74238542 0.25761458]
 [0.85917622 0.14082378]
 [0.78632428 0.21367572]
 [0.47211406 0.52788594]
 [0.82779151 0.17220849]
 [0.32448144 0.67551856]
 [0.23343309 0.76656691]
 [0.42004599 0.57995401]
 [0.67546812 0.32453188]
 [0.76684982 0.23315018]
 [0.4303539  0.5696461 ]
 [0.32934759 0.67065241]
 [0.50465741 0.49534259]
 [0.45957777 0.54042223]
 [0.71330635 0.28669365]
 [0.51115656 0.48884344]
 [0.67843497 0.32156503]
 [0.56158763 0.43841237]
 [0.47417502 0.52582498]
 [0.44508167 0.55491833]
 [0.56403211 0.43596789]
 [0.45956401 0.54043599]
 [0.79716306 0.20283694]
 [0.63675585 0.36324415]
 [0.67178079 0.32821921]
 [0.43557397 0.56442603]
 